# Creating an OSKAR MWA telescope

In [10]:
from subprocess import call
from astropy.io import fits
import numpy as np
from pathlib import Path


First of all, create the telescope dir.

In [11]:
Path.mkdir(Path("MWA_phase1"), exist_ok=True)

Next, write the array centre longitude, latitude and height.

In [4]:
with open("MWA_phase1/position.txt", 'w') as outfile:
    outfile.write("116.67081523611111 -26.703319405555554 377.827\n")

Get the `enh` coords and tile names of the MWA phase 1 layout from a metafits file. Write the locations into the `layout.txt` file.

In [12]:
metafits = '../../../examples/metafits/1126115208_metafits.fits'

with fits.open(metafits) as f:
    ##Need to order the antennas via the Tile column to be consistent
    ## with hyperdrive
    tilenames = f[1].data['Tile']

    ##metafits has positions for both X and Y pols which are identical so
    ##get every second position
    selection = np.arange(0, 256, 2)

    ##Get the east, north, height antenna positions from the metafits
    east = f[1].data['East'][selection]
    north = f[1].data['North'][selection]
    height = f[1].data['Height'][selection]
    tilenames = f[1].data['Tilename'][selection]
    
with open("MWA_phase1/layout.txt", 'w') as outfile:
    for e, n, h in zip(east, north, height):
        outfile.write(f"{e:.12f} {n:.12f} {h:.12f}\n")


Finally, we need to detail the station layout, which is an MWA tile. We could do this with a telescope-wide `station/layout.txt` file, but to link positions to tiles, I'll create stations named after each tile. TODO infact, this could be a hack to do missing dipoles/dipole amplitudes in the future?

In [20]:
##Spacing between dipoles is 1.1m on the ground mesh.
##Station layout coords are relative to station centre
##Farthest dipole centre from station centre is then (3*1.1)/2 = 1.65
##Assume flat layout (height = 0)


for tile in tilenames:
    Path.mkdir(Path(f"MWA_phase1/{tile}"), exist_ok=True)

    east_start = -1.65
    north_start = -1.65

    #os.mkdir('station')
    out_file = open(f'MWA_phase1/{tile}/layout.txt','w+')

    for i in range(4):
        for j in range(4):
            out_file.write('%.3f %.3f\n' %(east_start,north_start))
            east_start += 1.1
        east_start = -1.65
        north_start += 1.1

    out_file.close()

Make a really simple single point source at phase flat spectrum sky model.

In [22]:
ra = 0.0
dec = -26.703319405555554
flux_I = 1.0
flux_Q = 0.0
flux_U = 0.0
flux_V = 0.0
ref_freq = 180e6
SI = 0.0

with open("skymodel.osm", 'w') as outfile:
    outfile.write(f"{ra} {dec} {flux_I} {flux_Q} {flux_U} {flux_V} {ref_freq} {SI}\n")


Now we have the telescope and sky model, we can run `OSKAR`. Obviously this requires `OSKAR` to be installed. I've written a settings file `run_oskar_sim_interferometer.ini`, which I run via

```bash
$ oskar_sim_interferometer run_oskar_sim_interferometer.ini

```